In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
train_df = pd.read_json("./data/train.json")
test_df = pd.read_json("./data/test.json")
validation_df = pd.read_json("./data/validation.json")

In [8]:
train_df.head()

,sentence1,sentence2,label,guid,combined_sentences,all_tokens
0,Primul taragotist român a fost Nicolae Luță Io...,"Colegiul de arhitectură, artă și planificare (...",3,7cec5ac4-c115-4976-8d2a-9badfe9b63b9,Primul taragotist român a fost Nicolae Luță Io...,primul taragotist român a fost nicolae luță io...
1,Lupta revoluționarilor este condusă de Avram I...,Schiul nordic face parte din programul olimpic...,3,bc2fa29f-cf22-4a7c-8b55-9b1ed019f6ac,Lupta revoluționarilor este condusă de Avram I...,lupta revoluționarilor este condusă de avram i...
2,Locuitorii liberi au devenit „''iobagiones cas...,"În anii 1960, ea a apărut în drame realizate l...",3,8547b1ef-7bfe-43a9-aedf-bad0c0fbc049,Locuitorii liberi au devenit „''iobagiones cas...,locuitorii liberi au devenit iobagiones castri...
3,În anul 2002 are loc lansarea în domeniul turi...,Se lansează primul hotel al grupului în otopen...,2,0ad1ce19-7aa9-4ddd-b8d6-822072a723b0,În anul 2002 are loc lansarea în domeniul turi...,în anul are loc lansarea în domeniul turistich...
4,"Zillich a mijlocit, prin revista ''Klingsor'',...","Au apărut lucrări ale lui ion luca caragiale, ...",2,50c44ffa-b0c1-4d98-bc6c-3bbf95f50896,"Zillich a mijlocit, prin revista ''Klingsor'',...",zillich a mijlocit prin revista klingsor debut...


In [4]:
train_df['combined_sentences'] = train_df['sentence1'] + ' ' + train_df['sentence2']
test_df['combined_sentences'] = test_df['sentence1'] + ' ' + test_df['sentence2']
validation_df['combined_sentences'] = validation_df['sentence1'] + ' ' + validation_df['sentence2']

In [7]:
import re
from nltk.corpus import stopwords

sw = stopwords.words("romanian")

def text_preparetion(text):
    text = [i for i in
                re.split(r" +",  # 3
                     re.sub(r"[^a-zăâîșț@# ]", "",  # 2
                            text.lower()))]  # 1
    list_text_preprocessed = [word for word in text if word not in sw]  # 4
    list_text_preprocessed = ' '.join(list_text_preprocessed)
    return list_text_preprocessed

train_df['all_tokens'] = train_df['combined_sentences'].apply(text_preparetion)
test_df['all_tokens'] = test_df['combined_sentences'].apply(text_preparetion)
validation_df['all_tokens'] = validation_df['combined_sentences'].apply(text_preparetion)

In [9]:
X_train = train_df['all_tokens']
y_train = train_df['label']

In [10]:
X_val = validation_df['all_tokens']
y_val = validation_df['label']

In [11]:
X_test = test_df['all_tokens']

In [12]:
train_df.head()

,sentence1,sentence2,label,guid,combined_sentences,all_tokens
0,Primul taragotist român a fost Nicolae Luță Io...,"Colegiul de arhitectură, artă și planificare (...",3,7cec5ac4-c115-4976-8d2a-9badfe9b63b9,Primul taragotist român a fost Nicolae Luță Io...,primul taragotist român a fost nicolae luță io...
1,Lupta revoluționarilor este condusă de Avram I...,Schiul nordic face parte din programul olimpic...,3,bc2fa29f-cf22-4a7c-8b55-9b1ed019f6ac,Lupta revoluționarilor este condusă de Avram I...,lupta revoluționarilor este condusă de avram i...
2,Locuitorii liberi au devenit „''iobagiones cas...,"În anii 1960, ea a apărut în drame realizate l...",3,8547b1ef-7bfe-43a9-aedf-bad0c0fbc049,Locuitorii liberi au devenit „''iobagiones cas...,locuitorii liberi au devenit iobagiones castri...
3,În anul 2002 are loc lansarea în domeniul turi...,Se lansează primul hotel al grupului în otopen...,2,0ad1ce19-7aa9-4ddd-b8d6-822072a723b0,În anul 2002 are loc lansarea în domeniul turi...,în anul are loc lansarea în domeniul turistich...
4,"Zillich a mijlocit, prin revista ''Klingsor'',...","Au apărut lucrări ale lui ion luca caragiale, ...",2,50c44ffa-b0c1-4d98-bc6c-3bbf95f50896,"Zillich a mijlocit, prin revista ''Klingsor'',...",zillich a mijlocit prin revista klingsor debut...


In [13]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

In [14]:
X_test_tfidf = vectorizer.transform(X_test)

In [15]:
svm = SVC(kernel='linear')
svm.fit(X_train_tfidf, y_train)

SVC(kernel='linear')

In [16]:
import joblib

joblib.dump(svm, './models/svm_cu_tekenizare_ugraded.pkl')

['./models/svm_cu_tekenizare_ugraded.pkl']

In [12]:
import joblib
svm = joblib.load("./models/svm_cu_tekenizare.pkl")

In [17]:
val_predictions = svm.predict(X_val_tfidf)

In [19]:
from sklearn.metrics import classification_report

report = classification_report(y_val, val_predictions)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        74
           1       0.67      0.03      0.05        72
           2       0.54      0.74      0.62      1135
           3       0.78      0.65      0.71      1778

    accuracy                           0.66      3059
   macro avg       0.50      0.36      0.35      3059
weighted avg       0.67      0.66      0.65      3059


C:\Users\allex\miniconda3\envs\PyTorch\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\allex\miniconda3\envs\PyTorch\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\allex\miniconda3\envs\PyTorch\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
accuracy = accuracy_score(y_val, val_predictions)
print(f"Validation Accuracy: {accuracy}")

Validation Accuracy: 0.6433474991827395


In [34]:
y_test = svm.predict(X_test_tfidf)

In [35]:
result_df = pd.DataFrame({
    'guid': test_df['guid'],
    'label': y_test
})

In [38]:
result_df.head()

,guid,label
0,831cf870-c8e8-47f9-9318-7954706f08e3,3
1,4aabdcb9-9caa-4853-a0ec-d2767e365142,2
2,db383040-957a-475c-b8f7-5de7a36282f2,3
3,19d0dedc-04e1-4b7f-8061-2710f3bcbf86,2
4,3d293138-1740-4260-bd7e-64ab18d71e8d,3


In [36]:
result_df.to_csv("./data/submission3.csv", index=False)

In [37]:
test_df.size

15000